In [2]:
import pandas as pd
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [6]:
from experimentation import data_preparation
from experimentation import regression_OLS
from experimentation import backwardElimination